<p style="font-weight:bold;"> <span style="font-size: 36px"> Reimport of Transactional Data Test</span> </p>

In [0]:
#!eval-notebook "../Initialization/InitSystemorphBaseToMemory"

#  Definitions

## Args

In [0]:
var args = new ImportArgs("CH", 2021, 3, Periodicity.Quarterly, null, ImportFormats.Actual);
var previousArgs = new ImportArgs("CH", 2020, 12, Periodicity.Quarterly, null, ImportFormats.Actual);

var partitionReportingNode = new PartitionByReportingNode { Id = (Guid)(await DataSource.Partition.GetKeyForInstanceAsync<PartitionByReportingNode>(args)), ReportingNode = args.ReportingNode };

var reimportTrials = 5;

## DataNodeState

In [0]:
var dt11StatePrevious = new DataNodeState {
            DataNode = "DT1.1",
            State = State.Active,
            Year = previousArgs.Year,
            Month = previousArgs.Month,
            Partition = partitionReportingNode.Id
        };

var dtr11StatePrevious = new DataNodeState {
            DataNode = "DTR1.1",
            State = State.Active,
            Year = previousArgs.Year,
            Month = previousArgs.Month,
            Partition = partitionReportingNode.Id
        };

var dt11State = new DataNodeState {
            DataNode = "DT1.1",
            State = State.Active,
            Year = args.Year,
            Month = args.Month,
            Partition = partitionReportingNode.Id
        };

var dtr11State = new DataNodeState {
            DataNode = "DTR1.1",
            State = State.Active,
            Year = args.Year,
            Month = args.Month,
            Partition = partitionReportingNode.Id
        };

var dtr13StatePrevious = new DataNodeState {
            DataNode = "DTR1.3",
            State = State.Active,
            Year = previousArgs.Year,
            Month = previousArgs.Month,
            Partition = partitionReportingNode.Id
        };

var dt13State = new DataNodeState {
            DataNode = "DT1.3",
            State = State.Active,
            Year = args.Year,
            Month = args.Month,
            Partition = partitionReportingNode.Id
        };

var dtr13State = new DataNodeState {
            DataNode = "DTR1.3",
            State = State.Active,
            Year = args.Year,
            Month = args.Month,
            Partition = partitionReportingNode.Id
        };

var dt14State = new DataNodeState {
            DataNode = "DT1.4",
            State = State.Active,
            Year = args.Year,
            Month = args.Month,
            Partition = partitionReportingNode.Id
        };

var dtr14State = new DataNodeState {
            DataNode = "DTR1.4",
            State = State.Active,
            Year = args.Year,
            Month = args.Month,
            Partition = partitionReportingNode.Id
        };

# Case I: Reimport Openings for DataNode at Inception

## Disposable Workspace

In [0]:
var ws1 = Workspace.CreateNew();
ws1.Initialize(x=>x.FromSource(DataSource).DisableInitialization<DataNodeState>());
await ws1.UpdateAsync<DataNodeState>(new DataNodeState [] { //dt11StatePrevious with {State = State.Inactive},
                                                            //dtr11StatePrevious with {State = State.Inactive},
                                                            dt11State,
                                                            dtr11State,
                                                            // dtr13StatePrevious with {State = State.Inactive},
                                                            // dt13State with {State = State.Inactive},
                                                            // dtr13State with {State = State.Inactive},
                                                            // dt14State with {State = State.Inactive},
                                                            // dtr14State with {State = State.Inactive}
                                                          }
                                    );

In [0]:
var openings = @"
@@Main,,,,
ReportingNode,Year,Month,,
CH,2021,3,,
@@Opening,,,,
DataNode,EstimateType,AmountType,AccidentYear,Value
DT1.1,C,,,72.2
DT1.1,AA,PR,,-1.5
DT1.1,OA,PR,,1.5
";

## Import Phase

In [0]:
await Import.FromString(openings).WithFormat(ImportFormats.Opening).WithTarget(ws1).ExecuteAsync()

In [0]:
var caseIQueries = new { 
    IfrsVariables = ws1.Query<IfrsVariable>(), 
    RawVariables = ws1.Query<RawVariable>()
};

## Reimport Phase

In [0]:
for (int i=0; i < reimportTrials; i++)
{
    await Import.FromString(openings).WithFormat(ImportFormats.Opening).WithTarget(ws1).ExecuteAsync();
}

var caseIQueriesReimported = new { 
    IfrsVariables = ws1.Query<IfrsVariable>(), 
    RawVariables = ws1.Query<RawVariable>()
};

## Assertions

In [0]:
(   !caseIQueries.IfrsVariables.Except(caseIQueriesReimported.IfrsVariables, IfrsVariableComparer.Instance(ignoreValue: false)).Any() && 
    !caseIQueriesReimported.IfrsVariables.Except(caseIQueries.IfrsVariables, IfrsVariableComparer.Instance(ignoreValue: false)).Any()  ).Should().Be(true);

In [0]:
(   !caseIQueries.RawVariables.Except(caseIQueriesReimported.RawVariables, RawVariableComparer.Instance(ignoreValues: false)).Any() && 
    !caseIQueriesReimported.RawVariables.Except(caseIQueries.RawVariables, RawVariableComparer.Instance(ignoreValues: false)).Any()  ).Should().Be(true);

# Case II:  Reimport Actuals for DataNode at Inception and One Not at Inception

## Disposable Workspace

In [0]:
var ws2 = Workspace.CreateNew();
ws2.Initialize(x=>x.FromSource(DataSource).DisableInitialization<DataNodeState>());
await ws2.UpdateAsync<DataNodeState>(new DataNodeState [] { // dt11StatePrevious with {State = State.Inactive},
                                                            // dtr11StatePrevious with {State = State.Inactive},
                                                            dt11State,
                                                            dtr11State,
                                                            // dtr13StatePrevious with {State = State.Inactive},
                                                            dt13State,
                                                            // dtr13State with {State = State.Inactive},
                                                            // dt14State with {State = State.Inactive},
                                                            // dtr14State with {State = State.Inactive} 
                                                          }
                                    );

In [0]:
var actuals = @"
@@Main,,,,,
ReportingNode,Year,Month,,,
CH,2021,3,,,
@@Actual,,,,,
DataNode,AocType,AmountType,EstimateType,AccidentYear,Value
DT1.1,CF,PR,A,,90
DT1.1,CF,ICO,A,,-6
DT1.1,CF,NIC,A,,-70
DT1.1,CF,ACA,A,,-10
DT1.1,CF,AEA,A,,-5
DT1.3,CF,PR,A,,90
DT1.3,CF,ICO,A,,-6
DT1.3,CF,NIC,A,,-70
DT1.3,CF,ACA,A,,-10
DT1.3,CF,AEA,A,,-5";

## Import Phase

In [0]:
await Import.FromString(actuals).WithFormat(ImportFormats.Actual).WithTarget(ws2).ExecuteAsync()

In [0]:
var caseIIQueries = new { 
    IfrsVariables = ws2.Query<IfrsVariable>(), 
    RawVariables = ws2.Query<RawVariable>()
};

##  Reimport Phase

In [0]:
for (int i=0; i < reimportTrials; i++)
{
    await Import.FromString(actuals).WithFormat(ImportFormats.Actual).WithTarget(ws2).ExecuteAsync();
}

var caseIIQueriesReimported = new { 
    IfrsVariables = ws2.Query<IfrsVariable>(), 
    RawVariables = ws2.Query<RawVariable>()
};

## Assertions

In [0]:
(   !caseIIQueries.IfrsVariables.Except(caseIIQueriesReimported.IfrsVariables, IfrsVariableComparer.Instance(ignoreValue: false)).Any() && 
    !caseIIQueriesReimported.IfrsVariables.Except(caseIIQueries.IfrsVariables, IfrsVariableComparer.Instance(ignoreValue: false)).Any()  ).Should().Be(true);

In [0]:
(   !caseIIQueries.RawVariables.Except(caseIIQueriesReimported.RawVariables, RawVariableComparer.Instance(ignoreValues: false)).Any() && 
    !caseIIQueriesReimported.RawVariables.Except(caseIIQueries.RawVariables, RawVariableComparer.Instance(ignoreValues: false)).Any()  ).Should().Be(true);

# Case III: Openings for Insurance at Inception and Cashflows for Linked Preexisitng Reinsurance Not at Inception

## Disposable Workspace

In [0]:
var ws3 = Workspace.CreateNew();
ws3.Initialize(x=>x.FromSource(DataSource).DisableInitialization<DataNodeState>());
await ws3.UpdateAsync<DataNodeState>(new DataNodeState [] { // dt11StatePrevious with {State = State.Inactive},
                                                            // dtr11StatePrevious with {State = State.Inactive},
                                                            // dt11State with {State = State.Inactive},
                                                            // dtr11State with {State = State.Inactive},
                                                            dtr13StatePrevious,
                                                            // dt13State with {State = State.Inactive},
                                                            // dtr13State with {State = State.Inactive},
                                                            dt14State,
                                                            // dtr14State with {State = State.Inactive}
                                                          }
                                    );

In [0]:
var actuals = @"
@@Main,,,,,
ReportingNode,Year,Month,,,
CH,2021,3,,,
@@Actual,,,,,
DataNode,AocType,AmountType,EstimateType,AccidentYear,Value
DT1.4,CF,PR,A,,90
DT1.4,CF,ICO,A,,-6
DT1.4,CF,NIC,A,,-70
DT1.4,CF,ACA,A,,-10
DT1.4,CF,AEA,A,,-5";

var openings = @"
@@Main
ReportingNode,Year,Month,,
CH,2021,3,,
@@Opening
DataNode,EstimateType,AmountType,AccidentYear,Value
DT1.4,C,,,72.2
DT1.4,AA,PR,,-1.5
DT1.4,OA,PR,,1.5";

var cashflows = @"
@@Main,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
ReportingNode,Year,Month,Scenario,,,,,,,,,,,,,,,,,,,,,,,,,,
CH,2021,3,,,,,,,,,,,,,,,,,,,,,,,,,,,
@@Cashflow,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
DataNode,AmountType,EstimateType,AocType,Novelty,AccidentYear,Values0,Values1,Values2,Values3,Values4,Values5,Values6,Values7,Values8,Values9,Values10,Values11,Values12,Values13,Values14,Values15,Values16,Values17,Values18,Values19,Values20,Values21,Values22,Values23
DTR1.3,PR,BE,BOP,N,,100,0,0,100,0,0,100,0,0,100,0,0,0,100,0,0,100,0,0,100,0,0,100,0
DTR1.3,NIC,BE,BOP,N,,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25
DTR1.3,,CU,BOP,N,,-10,-10,-10,-10,-10,-10,-10,-10,-10,-10,-10,-10,-10,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-3
DTR1.3,,RA,BOP,N,,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5
DTR1.3,PR,BE,CL,C,,100,0,0,100,0,0,100,0,0,100,0,0,0,100,0,0,100,0,0,100,0,0,100,0
DTR1.3,NIC,BE,CL,C,,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25
DTR1.3,,CU,CL,C,,-10,-10,-10,-10,-10,-10,-10,-10,-10,-10,-10,-10,-10,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-3
DTR1.3,,RA,CL,C,,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5
";

## Import Sequence Phase

In [0]:
var ws31 = Workspace.CreateNew();
ws31.Initialize(x=>x.FromSource(ws3));
await Import.FromString(actuals).WithFormat(ImportFormats.Actual).WithTarget(ws31).ExecuteAsync();
await Import.FromString(actuals).WithFormat(ImportFormats.Actual).WithTarget(ws3).ExecuteAsync()

In [0]:
var caseIIIQuery1= new { 
    IfrsVariables = ws31.Query<IfrsVariable>(), 
    RawVariables = ws31.Query<RawVariable>()
};

In [0]:
var ws32 = Workspace.CreateNew();
ws32.Initialize(x=>x.FromSource(ws3).DisableInitialization<IfrsVariable>().DisableInitialization<RawVariable>());
await Import.FromString(openings).WithFormat(ImportFormats.Opening).WithTarget(ws32).ExecuteAsync();
await Import.FromString(openings).WithFormat(ImportFormats.Opening).WithTarget(ws3).ExecuteAsync()

In [0]:
var caseIIIQuery2 = new { 
    IfrsVariables = ws32.Query<IfrsVariable>(), 
    RawVariables = ws32.Query<RawVariable>()
};

In [0]:
var ws33 = Workspace.CreateNew();
ws33.Initialize(x=>x.FromSource(ws3).DisableInitialization<IfrsVariable>().DisableInitialization<RawVariable>());
await Import.FromString(cashflows).WithFormat(ImportFormats.Cashflow).WithTarget(ws33).ExecuteAsync();
await Import.FromString(cashflows).WithFormat(ImportFormats.Cashflow).WithTarget(ws3).ExecuteAsync()

In [0]:
var caseIIIQuery3 = new { 
    IfrsVariables = ws33.Query<IfrsVariable>(), 
    RawVariables = ws33.Query<RawVariable>()
};

In [0]:
var caseIIIQueries = new { 
    IfrsVariables = ws3.Query<IfrsVariable>(), 
    RawVariables = ws3.Query<RawVariable>()
};

## Assertions

<HR>
/NOTES 

EXCEPT(... , **Raw**VariableComparer.Instance(ignoreValue**s**: false) and 
EXCEPT(... , *Ifrs*VariableComparer.Instance(ignoreValue: false))

can be used to compare two queries from the workspace.

In [0]:
caseIIIQueries.IfrsVariables.Except(
                                        caseIIIQuery1.IfrsVariables.Union(caseIIIQuery2.IfrsVariables
                                                                   .Union(caseIIIQuery3.IfrsVariables))
                                        , IfrsVariableComparer.Instance(ignoreValue: false))

In [0]:
caseIIIQueries.RawVariables.Except(
                                        caseIIIQuery1.RawVariables.Union(caseIIIQuery2.RawVariables
                                                                  .Union(caseIIIQuery3.RawVariables)), RawVariableComparer.Instance(ignoreValues: false)
                                        )

In [0]:
// caseIIIQueries.RawVariables.SequenceEqual(caseIIIQuery1.RawVariables.Union(caseIIIQuery2.RawVariables.Union(caseIIIQuery3.RawVariables)))

The assertion below shouldn't fail...

NOTES/
<HR>

In [0]:
(   !caseIIIQueries.IfrsVariables.Except(
                                        caseIIIQuery1.IfrsVariables.Union(caseIIIQuery2.IfrsVariables
                                                                   .Union(caseIIIQuery3.IfrsVariables)),
                                        IfrsVariableComparer.Instance(ignoreValue: false)
                                        ).Any() && 
    !caseIIIQueries.RawVariables.Except(
                                        caseIIIQuery1.RawVariables.Union(caseIIIQuery2.RawVariables
                                                                  .Union(caseIIIQuery3.RawVariables)),
                                        RawVariableComparer.Instance(ignoreValues: false)
                                        ).Any()
).Should().Be(true);

# Case V: Reimport Modified Openings with CSM Deletion and LC Addition for the Same DataNode

## Disposable Workspace

In [0]:
var ws4 = Workspace.CreateNew();
ws4.Initialize(x=>x.FromSource(DataSource).DisableInitialization<DataNodeState>());
await ws4.UpdateAsync<DataNodeState>(new DataNodeState [] { //dt11StatePrevious with {State = State.Inactive},
                                                            //dtr11StatePrevious with {State = State.Inactive},
                                                            dt11State,
                                                            dtr11State,
                                                            // dtr13StatePrevious with {State = State.Inactive},
                                                            // dt13State with {State = State.Inactive},
                                                            // dtr13State with {State = State.Inactive},
                                                            // dt14State with {State = State.Inactive},
                                                            // dtr14State with {State = State.Inactive}
                                                          }
                                    );

In [0]:
var openings = @"
@@Main,,,,
ReportingNode,Year,Month,,
CH,2021,3,,
@@Opening,,,,
DataNode,EstimateType,AmountType,AccidentYear,Value
DT1.1,C,,,72.2
DT1.1,AA,PR,,-1.5
DT1.1,OA,PR,,1.5
";

## Import Sequence Phase

In [0]:
var ws41 = Workspace.CreateNew();
ws41.Initialize(x=>x.FromSource(ws4));
await Import.FromString(openings).WithFormat(ImportFormats.Opening).WithTarget(ws41).ExecuteAsync();
await Import.FromString(openings).WithFormat(ImportFormats.Opening).WithTarget(ws4).ExecuteAsync()

In [0]:
var caseVQuery1 = new { 
    IfrsVariables = ws41.Query<IfrsVariable>(), 
    RawVariables = ws41.Query<RawVariable>()
};

In [0]:
var openings = @"
@@Main,,,,
ReportingNode,Year,Month,,
CH,2021,3,,
@@Opening,,,,
DataNode,EstimateType,AmountType,AccidentYear,Value
DT1.1,L,,,72.2
DT1.1,AA,PR,,-1.5
DT1.1,OA,PR,,1.5
";

In [0]:
var ws42 = Workspace.CreateNew();
ws42.Initialize(x=>x.FromSource(ws4).DisableInitialization<IfrsVariable>().DisableInitialization<RawVariable>());
await Import.FromString(openings).WithFormat(ImportFormats.Opening).WithTarget(ws42).ExecuteAsync();
await Import.FromString(openings).WithFormat(ImportFormats.Opening).WithTarget(ws4).ExecuteAsync()

In [0]:
var caseVQuery2 = new { 
    IfrsVariables = ws42.Query<IfrsVariable>(), 
    RawVariables = ws42.Query<RawVariable>()
};

In [0]:
var caseVQueries = new { 
    IfrsVariables = ws4.Query<IfrsVariable>(), 
    RawVariables = ws4.Query<RawVariable>()
};

## Assertions

In [0]:
(   !caseVQueries.IfrsVariables.Except(
                                        caseVQuery1.IfrsVariables.Union(caseVQuery2.IfrsVariables),
                                        IfrsVariableComparer.Instance(ignoreValue: false)
                                      ).Any() && 
    !caseVQueries.RawVariables.Except(
                                        caseVQuery1.RawVariables.Union(caseVQuery2.RawVariables),
                                        RawVariableComparer.Instance(ignoreValues: false)
                                     ).Any()
).Should().Be(true);